In [9]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import pickle
import time
import os
import shutil
import shap
import seaborn as sns
from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import LambdaLR, LinearLR, SequentialLR
from functools import partial
from sklearn.metrics import f1_score, accuracy_score
from sklearn.calibration import calibration_curve


from codes.models.metrics import calculate_roc_auc
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList, Patient
from codes.models.Transformers.dic_model_versions import DIC_MODEL_VERSIONS
from codes.tests.TestsClass import TestSet, TrainTransformerModel, TrainModel
import matplotlib.pyplot as plt
from codes.models.Decision_tree.utils import get_name, get_indice



In [10]:
model_type = 'Generative_Transformer'
model_version = 'V1'
test_name = 'tests_generative_phenos_2'
CHR = 1
SNP = 'rs673604'
pheno_method = 'Paul' # Paul, Abby
rollup_depth = 4

In [15]:
CHR = 1
SNP = 'rs673604'

path = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/'
#check test name
model_dir = path + f'logs/runs/SNPS/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}'
model_plot_dir = path + f'logs/plots/tests/SNP/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}/'

os.makedirs(model_dir, exist_ok=True)
os.makedirs(model_plot_dir, exist_ok=True)
#check number tests
test_dir = f'{model_dir}/{test_name}/'
print(test_dir)
log_data_dir = f'{test_dir}/data/'
log_tensorboard_dir = f'{test_dir}/tensorboard/'
log_slurm_outputs_dir = f'{test_dir}/Slurm/Outputs/'
log_slurm_errors_dir = f'{test_dir}/Slurm/Errors/'
os.makedirs(log_data_dir, exist_ok=True)
os.makedirs(log_tensorboard_dir, exist_ok=True)
os.makedirs(log_slurm_outputs_dir, exist_ok=True)
os.makedirs(log_slurm_errors_dir, exist_ok=True)


log_data_path_pickle = f'{test_dir}/data/{test_name}.pkl'
log_tensorboard_path = f'{test_dir}/tensorboard/{test_name}'
log_slurm_outputs_path = f'{test_dir}/Slurm/Outputs/{test_name}.txt'
log_slurm_error_path = f'{test_dir}/Slurm/Errors/{test_name}.txt'
model_plot_path = path + f'logs/plots/tests/SNP/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}/{test_name}.png'


/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/logs/runs/SNPS/1/rs673604/Generative_Transformer/V1/Paul/tests_generative_phenos_2/


In [16]:
with open(log_data_path_pickle, 'rb') as file:
    model = pickle.load(file)

In [17]:
model

PhenotypeEncodingAlone(
  (projection_embed): Linear(in_features=10, out_features=10, bias=True)
  (blocks): PadMaskSequential(
    (0): BlockPheno(
      (sa): MultiHeadSelfAttention(
        (qkv_network): Linear(in_features=10, out_features=12, bias=False)
        (proj): Linear(in_features=4, out_features=10, bias=True)
        (attention_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=10, out_features=40, bias=True)
          (1): ReLU()
          (2): Linear(in_features=40, out_features=10, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
    )
    (1): BlockPheno(
      (sa): MultiHeadSelfAttention(
        (qkv_network): Linear(in_features=10, out_features=12, bias=False)
        (pr